<a href="https://colab.research.google.com/github/AanLetna7025/WEB_bot/blob/main/toobotFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playwright langchain langchain-community langchain-huggingface sentence-transformers google-generativeai nest_asyncio faiss-cpu torch

  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.7.0
    Uninstalling google-ai-generativelanguage-0.7.0:
      Successfully uninstalled google-ai-generativelanguage-0.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.12 requires google-ai-generativelanguage<1,>=0.7, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [ ]:
!pip install langchain_google_genai

  Using cached google_ai_generativelanguage-0.7.0-py3-none-any.whl.metadata (10 kB)
Using cached google_ai_generativelanguage-0.7.0-py3-none-any.whl (1.4 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.


In [ ]:
!playwright install

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.12/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:269:9)
    at async Registry._validateHostRequirements (/usr/local/lib/python3.12/dist

need not run this cell,go down.

In [ ]:
import asyncio
import nest_asyncio
from urllib.parse import urlparse
from playwright.async_api import async_playwright
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import torch
import gc
nest_asyncio.apply()

# Check GPU
print(f" GPU Available: {' T4 GPU' if torch.cuda.is_available() else ' CPU Only'}")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Setup
from google.colab import drive, userdata
drive.mount('/content/drive')
api_key = userdata.get('gemini_key')

async def quick_scrape(url):
    """Ultra-fast scraping"""
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True, args=['--no-sandbox', '--disable-dev-shm-usage'])
        page = await browser.new_page()
        await page.route("**/*.{png,jpg,gif,css,woff}", lambda route: route.abort())

        try:
            # Wait for DOM content to load and then wait for an additional 3 seconds for JS to execute
            await page.goto(url, timeout=20000, wait_until='domcontentloaded')
            await page.wait_for_timeout(3000)


            content = await page.evaluate("""
                () => {
                    ['script', 'style', 'nav', 'footer', 'header'].forEach(tag => {
                        document.querySelectorAll(tag).forEach(el => el.remove());
                    });
                    return document.body.innerText.replace(/\\s+/g, ' ').trim();
                }
            """)

            links = await page.evaluate("""
                () => Array.from(document.querySelectorAll('a[href]'))
                    .map(a => a.href)
                    .filter(href => href.startsWith(window.location.origin))
            """)

        except Exception as e:
            print(f" Error: {url} - {e}")
            # Return None for content and links to indicate an error
            return None, None

        await browser.close()
        return content, links

async def fast_crawl(root_url="https://www.toobler.com/"):
    """Lightning-fast crawling"""
    visited = set()
    to_visit = [root_url]
    docs = []

    while to_visit:
        # Create a set of URLs to scrape in the current batch from to_visit
        current_batch_urls = set()
        while to_visit and len(current_batch_urls) < 3:
            url = to_visit.pop(0) # Use pop(0) for breadth-first
            if url not in visited:
                current_batch_urls.add(url)

        if not current_batch_urls:
            continue

        tasks = [quick_scrape(url) for url in current_batch_urls]
        results = await asyncio.gather(*tasks, return_exceptions=True)

        for i, result in enumerate(results):
            if isinstance(result, Exception):
                # Handle the exception, e.g., print an error message
                url = list(current_batch_urls)[i] # Get the URL corresponding to the task
                print(f" Failed to scrape {url}: {result}")
                continue # Skip this result and move to the next

            content, links = result
            if content is not None and len(content) > 200:
                url = list(current_batch_urls)[i] # Get the URL corresponding to the task
                visited.add(url)
                docs.append(Document(page_content=content, metadata={"source": url}))
                print(f" {len(docs)}: {url[:50]}...")

                # Add new links to to_visit if not visited or already in to_visit
                if links:
                    for link in links:
                         # Use urlparse to normalize URLs before checking visited/to_visit
                        normalized_link = urlparse(link)._replace(fragment='').geturl() # Remove fragments
                        if normalized_link not in visited and normalized_link not in to_visit:
                             to_visit.append(normalized_link)


    print(f"🎉 Scraped {len(docs)} pages")
    return docs


def create_gpu_vectorstore(docs):
    """GPU-accelerated vector store creation"""
    print(" Creating AI knowledge base...")

    # GPU-optimized embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': device},
        encode_kwargs={'normalize_embeddings': True}
    )

    # Quick chunking
    splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)
    chunks = splitter.split_documents(docs)
    # Limit chunks to a reasonable number to avoid hitting resource limits during vector store creation
    # For 107 pages, 200 chunks might be too low. Let's increase the limit or remove it.
    # Removing the limit for now, assuming 107 pages won't create an excessive number of chunks.
    chunks = [c for c in chunks if len(c.page_content.strip()) > 100]


    print(f" Processing {len(chunks)} chunks on {device.upper()}...")

    # Create vector store (GPU accelerated if available)
    vectorstore = FAISS.from_documents(documents=chunks, embedding=embeddings)


    # Save
    save_path = "/content/drive/MyDrive/toobler_vectorstore"
    vectorstore.save_local(save_path)
    print(f"💾 Saved to {save_path}")

    return vectorstore

def setup_qa():
    """Setup Q&A system"""
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': device},
        encode_kwargs={'normalize_embeddings': True}
    )

    vectorstore = FAISS.load_local(
        "/content/drive/MyDrive/toobler_vectorstore",
        embeddings,
        allow_dangerous_deserialization=True
    )

    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=api_key, temperature=0.1)

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an assistant that answers questions strictly based on the provided context. "
        "If the answer is not contained within the context, respond with: 'I cannot find the answer in the provided information.' "
        "Only provide relevant, concise answers. Do not add any information not supported by the context. "
        "Avoid hallucinations and do not speculate."),
        ("human", "Context:\n{context}\n\nQuestion:\n{question}\n\nAnswer:")
    ])

    chain = prompt | llm | StrOutputParser()

    def ask(question):
        docs = vectorstore.similarity_search(question, k=3)
        context = "\n".join([doc.page_content for doc in docs])
        response = chain.invoke({"context": context, "question": question})
        print(f"🤖 {response}")
        return response

    return ask

# MAIN EXECUTION - One command does everything!
async def main():
    print(" SCRAPER")
    print("=" * 40)

    # 1. Scrape (2-3 minutes)
    docs = await fast_crawl()
    if not docs:
        print(" No content found!")
        return

    # 2. Create knowledge base (30-60 seconds on GPU)
    vectorstore = create_gpu_vectorstore(docs)

    # 3. Ready for questions!
    ask = setup_qa()
    print("\n READY! Ask questions about Toobler:")

    return ask

# RUN EVERYTHING
ask_function = await main()

# Interactive mode (optional)
print("\n" + "="*50)
print(" INTERACTIVE MODE - Ask your own questions!")
while True:
    q = input("\n Question (or 'quit'): ")
    if q.lower() == 'quit':
        break
    if q.strip():
        ask_function(q)

 GPU Available:  T4 GPU
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 SCRAPER
 1: https://www.toobler.com/...
 2: https://www.toobler.com/about-us...
 3: https://www.toobler.com/casestudy/iot-based-invent...
 4: https://www.toobler.com/contact...
 5: https://www.toobler.com/casestudy/reduce-downtime-...
 6: https://www.toobler.com/casestudy/building-worlds-...
 7: https://www.toobler.com/services/App-development-f...
 8: https://www.toobler.com/blog/iot-fleet-management...
 9: https://www.toobler.com/blog/digital-twin-examples...
 10: https://www.toobler.com/blog/esim-vs-isim-iot...
 11: https://www.toobler.com/blog/manufacturers-guide-t...
 12: https://www.toobler.com/casestudy/finance-applicat...
 13: https://www.toobler.com/blog...
 14: https://www.toobler.com/casestudy/mobile-app-regat...
 15: https://www.toobler.com/casestudy...
 16: https://www.toobler.com/casestudy/casestudy...
 17: https://www.t

In [ ]:
def setup_qa():
    """Setup Q&A system"""
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': device},
        encode_kwargs={'normalize_embeddings': True}
    )

    # This is the code to load the vector store from Google Drive
    load_path = "/content/drive/MyDrive/toobler_vectorstore"
    vectorstore = FAISS.load_local(
        load_path,
        embeddings,
        allow_dangerous_deserialization=True
    )

    # The rest of the setup_qa function to create the Q&A chain
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=api_key, temperature=0.1)

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an assistant that answers questions strictly based on the provided context. "
        "If the answer is not contained within the context, respond with: 'I cannot find the answer in the provided information.' "
        "Only provide relevant, concise answers. Do not add any information not supported by the context. "
        "Avoid hallucinations and do not speculate."),
        ("human", "Context:\n{context}\n\nQuestion:\n{question}\n\nAnswer:")
    ])

    chain = prompt | llm | StrOutputParser()

    def ask(question):
        docs = vectorstore.similarity_similarity_search(question, k=3)
        context = "\n".join([doc.page_content for doc in docs])
        response = chain.invoke({"context": context, "question": question})
        print(f"🤖 {response}")
        return response

    return ask